In [1]:
import os
os.chdir('../sheaf_kg')
import sys
import time
import argparse
import json
from tqdm import tqdm

import pandas as pd
import numpy as np
import pykeen
from pykeen.evaluation import rank_based_evaluator
import torch
from scipy import stats

from sheaf_kg.train_sheafE_betae import read_dataset, shuffle_datasets, dataset_to_device, sample_answers
import sheaf_kg.complex_functions as cf
# from sheaf_kg.complex_functions import test_batch

PyKEEN was unable to load dataset openbiolinkf1. Try uninstalling PyKEEN with ``pip uninstall pykeen`` then reinstalling
PyKEEN was unable to load dataset openbiolinkf2. Try uninstalling PyKEEN with ``pip uninstall pykeen`` then reinstalling


In [2]:
dataset = 'FB15k-237'
savenames = ['SheafE_Translational_64embdim_64esdim_1sec_2norm_0.0lbda_250epochs_MarginRankingLossloss_None_11seed_20210527-1849',
            'SheafE_Translational_64embdim_64esdim_1sec_2norm_0.0lbda_250epochs_MarginRankingLossloss_None_11seed_20210527-1913',
            'SheafE_Translational_64embdim_64esdim_1sec_2norm_0.0lbda_250epochs_MarginRankingLossloss_None_20210525-1447',
            ]
savedir = f'/home/gebhart/projects/sheaf_kg/data/{dataset}'
complex_dataset_loc = f'/home/gebhart/projects/sheaf_kg/data/{dataset}-betae'

test_query_structures = ['1p','2p','3p','2i','3i','ip','pi']


In [3]:
fun_map = {'1p':cf.L_p_traversal_transE,
           '2p':cf.L_p_traversal_transE,
           '3p':cf.L_p_traversal_transE,
           '2i':cf.L_i_traversal_transE,
           '3i':cf.L_i_traversal_transE,
           'ip':cf.L_ip_traversal_transE,
           'pi':cf.L_pi_traversal_transE
          }

def test_batch(model, test_data, model_inverses=False, test_batch_size=5,
                test_query_structures=test_query_structures,
                ks=[1,3,5,10]):
    with torch.no_grad():
        results = []
        for query_structure in test_query_structures:
            print('Running query : {}'.format(query_structure))
            all_avg_ranks = []
            cnt = 0
            num_test = len(test_data[query_structure]['answers'])
            for qix in tqdm(range(0, num_test, test_batch_size)):
                if num_test - qix == 1:
                    continue
                entities = test_data[query_structure]['entities'][qix:qix+test_batch_size]
                relations = test_data[query_structure]['relations'][qix:qix+test_batch_size]
                if model_inverses:
                    inverses = None
                else:
                    inverses = test_data[query_structure]['inverses'][qix:qix+test_batch_size]
                all_answers = test_data[query_structure]['answers'][qix:qix+test_batch_size]
                targets = torch.arange(model.num_entities).to(model.device)
                Q = fun_map[query_structure](model, entities, relations, targets, invs=inverses, p=2, variety='sheafE')
                answer_lens = np.array([len(a) for a in all_answers])
                max_len = answer_lens.max()
                for l in np.unique(answer_lens):
                    idxs = np.where(answer_lens == l)[0]
                    answers = [all_answers[j] for j in idxs]
                    filter_fix = torch.arange(l)
                    for aix in range(len(idxs)):
                        ranks = rank_based_evaluator.compute_rank_from_scores(Q[idxs[aix],answers[aix]].unsqueeze(1), Q[idxs[aix],:].unsqueeze(0))
                        avg_rank = (torch.sort(ranks['realistic'].cpu(), dim=0)[0] - filter_fix).numpy()
                        all_avg_ranks.append(avg_rank)
            all_avg_ranks = np.concatenate(all_avg_ranks)
            rd = {k: np.mean(all_avg_ranks <= k) for k in ks}
            mrr = np.reciprocal(stats.hmean(all_avg_ranks))
            rd['mrr'] = mrr if isinstance(mrr, float) else mrr[0]
            # rd['mr'] = np.mean(all_avg_ranks)
            results.append(rd)

        df = pd.DataFrame(results, index=test_query_structures)
        return df

In [4]:
datasets = read_dataset(complex_dataset_loc)
datasets = dataset_to_device(shuffle_datasets(datasets), 'cuda')

In [5]:
results = []
for savename in savenames:
    model = torch.load(os.path.join(savedir, savename, 'trained_model.pkl'))
    extension_df = test_batch(model, datasets['test-easy'], model_inverses=True, test_query_structures=test_query_structures)
    results.append(extension_df)    

  1%|▏         | 47/3262 [00:00<00:06, 469.23it/s]

Running query : 1p


  2%|▏         | 19/977 [00:00<00:05, 188.88it/s]

Running query : 2p


  2%|▏         | 19/974 [00:00<00:05, 184.81it/s]

...

  3%|▎         | 24/909 [00:00<00:03, 233.53it/s]

Running query : ip


  4%|▍         | 33/857 [00:00<00:02, 325.11it/s]

Running query : pi


100%|██████████| 857/857 [00:02<00:00, 334.84it/s]


In [6]:
mndf = sum(results)/len(results)
mndf*100

,1,3,5,10,mrr
1p,4.807813,11.499428,15.435972,21.397169,10.157203
2p,0.587921,1.310348,1.865469,2.856724,1.435568
3p,0.277180,0.606934,0.869706,1.392268,0.753042
2i,0.495210,1.498202,2.431972,4.406155,1.959023
3i,0.390084,1.151267,1.961680,4.041688,1.895674
ip,0.418201,0.930820,1.458903,2.824464,1.288238
pi,0.235500,0.705599,1.231033,2.716474,1.317666


In [7]:
print((mndf.transpose().round(4)*100).to_latex())

\begin{tabular}{lrrrrrrr}
\toprule
{} &     1p &    2p &    3p &    2i &    3i &    ip &    pi \\
\midrule
1   &   4.81 &  0.59 &  0.28 &  0.50 &  0.39 &  0.42 &  0.24 \\
3   &  11.50 &  1.31 &  0.61 &  1.50 &  1.15 &  0.93 &  0.71 \\
5   &  15.44 &  1.87 &  0.87 &  2.43 &  1.96 &  1.46 &  1.23 \\
10  &  21.40 &  2.86 &  1.39 &  4.41 &  4.04 &  2.82 &  2.72 \\
mrr &  10.16 &  1.44 &  0.75 &  1.96 &  1.90 &  1.29 &  1.32 \\
\bottomrule
\end{tabular}

